# load packages

In [ ]:
import pandas as pd

In [ ]:
import requests

In [ ]:
import sys

# read in input files

## go database

In [ ]:
go = pd.read_csv('pathway_score/pathway_annotation/raw_databases/goa_human.gaf.gz',
                 sep = '\t',
                 comment = '!',
                 header = None,
                 low_memory = False)
go.head()

In [ ]:
print(len(go.index))
print(len(go[10].unique()))
print(len(go[4].unique()))
print(len(go[9].unique()))
print(go[3].unique())

In [ ]:
go.sort_values(by = [4, 9]).head()

## gene lists

In [ ]:
meta_gene_list = pd.read_csv('common_var_gene_score/igap_adsp_gene_score/merged_outputs/AOU_ALL.UKBB.metasoft.ADSP.all.VEP_v113.gene_by_position.r2_0.1_clump_variants_excluded.RE_pval_threshold_0.05.gene_score_list.txt')
print(len(meta_gene_list.index))
meta_gene_list.head()

### rnaseq gene list

In [ ]:
rnaseq_gene = pd.read_csv('pathway_score/rosmap/ROSMAP_RNAseq_genes.txt',
                        header = None)
print(len(rnaseq_gene.index))
rnaseq_gene.head()

In [ ]:
msbb_rnaseq_gene = pd.read_csv('pathway_score/msbb/MSBB.RNAseq.BA_ALL.19batch.gene_symbol.individualID.mvalue_norm.tpm.log2.coding_genes_only.covar_corrected.gene_list.txt')
print(len(msbb_rnaseq_gene.index))
msbb_rnaseq_gene.head()

### methylation gene list

In [ ]:
methyl_gene = pd.read_csv('pathway_score/rosmap/ROSMAP_arrayMethylation_imputed.gene_list.txt')
print(len(methyl_gene.index))
methyl_gene.head()

In [ ]:
msbb_methyl_gene = pd.read_csv('pathway_score/msbb/MSBB.methylation_array.19batch.gene_symbol.individualID.mvalue_norm.gene_list.txt')
print(len(msbb_methyl_gene.index))
msbb_methyl_gene.head()

### tmt proteomics gene list

In [ ]:
msbb_tmt_gene = pd.read_csv('pathway_score/msbb/MSBB.TMT_proteomics.19batch.normalized.gene_symbol.individualID.log2_transformed.gene_list.txt')
print(len(msbb_tmt_gene.index))
msbb_tmt_gene.head()

### somoscan proteomics

In [ ]:
somoscan_gene = pd.read_csv('pathway_score/rosmap/ROSMAP.proteomics.somoscan.individualID.entrez_gene_symbol.gene_list.txt')
print(len(somoscan_gene.index))
somoscan_gene.head()

## gene map

In [ ]:
gene_map = pd.read_csv('/project/ritchie/projects/ADSP_Projects/ADSP_Annotations/VEP_annotation_manual_113/ensembl_start_stop/Homo_sapiens.GRCh38.113.gene_start_stop.autosomes.500kb_upstream_downstream.gtf.txt',
                       sep = '\t')
gene_map.head()

## ref genes

In [ ]:
ref = pd.read_csv('/project/ritchie/projects/ADSP_Projects/ADSP_Annotations/VEP_annotation_manual_113/ensembl_start_stop/Homo_sapiens.GRCh38.113.refseq.exp_validated.gene_map.csv')
ref.head()

# clean pathway files

## remove NOT associations

In [ ]:
go_clean = go[~go[3].str.contains('NOT')]
print(len(go.index))
print(len(go_clean.index))
print(go_clean[3].unique())

## remove special characters from gene col

In [ ]:
go_clean[10] = go_clean[10].str.replace('[', '')
go_clean[10] = go_clean[10].str.replace(']', '')
go_clean[10] = go_clean[10].str.replace(')', '')
go_clean[10] = go_clean[10].str.replace('(', ' ')
go_clean[10] = go_clean[10].str.replace("'", '')

## explode gene

In [ ]:
print(len(go_clean.index))
go_clean = (go_clean.assign(GENE = go[10].str.split('|')).explode('GENE').reset_index(drop = True))
print(len(go_clean.index))
go_clean = (go_clean.assign(GENE = go_clean['GENE'].str.split(':')).explode('GENE').reset_index(drop = True))
print(len(go_clean.index))
go_clean = (go_clean.assign(GENE = go_clean['GENE'].str.split(';')).explode('GENE').reset_index(drop = True))
print(len(go_clean.index))
go_clean = (go_clean.assign(GENE = go_clean['GENE'].str.split(',')).explode('GENE').reset_index(drop = True))
print(len(go_clean.index))
go_clean = (go_clean.assign(GENE = go_clean['GENE'].str.split('/')).explode('GENE').reset_index(drop = True))
print(len(go_clean.index))
go_clean = (go_clean.assign(GENE = go_clean['GENE'].str.split(' ')).explode('GENE').reset_index(drop = True))
print(len(go_clean.index))

## subset and rename

In [ ]:
go_clean = go_clean[[4, 9, 'GENE', 3]]
go_clean = go_clean.rename(columns = {4 : 'PATHWAY_ID', 9 : 'PATHWAY_NAME', 3 : 'RELATION'})
go_clean = go_clean[['PATHWAY_ID', 'PATHWAY_NAME', 'GENE', 'RELATION']]
go_clean = go_clean.dropna().drop_duplicates()
print(len(go_clean.index))
go_clean.head()

In [ ]:
print(len(go_clean.index))
print(len(go_clean['PATHWAY_ID'].unique()))

In [ ]:
go_clean[go_clean['PATHWAY_ID'].duplicated(keep = False)].sort_values(by = 'PATHWAY_ID')

In [ ]:
print(len(go_clean.drop(columns = ['PATHWAY_NAME', 'RELATION']).drop_duplicates().index))
print(len(go_clean.drop(columns = ['RELATION']).drop_duplicates().index))
print(len(go_clean.index))

## fix typos

In [ ]:
go_clean['GENE'] = go_clean['GENE'].str.replace('^2 x ', '2x', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('F3x', 'F 3x', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('B1x2x', 'B 2x', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace(r'GPIA\*', 'GPIA', regex = True)

# remove "2x" symbols
go_clean['GENE'] = go_clean['GENE'].str.replace('^1x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^2x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^3x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^4x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^5x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^6x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^7x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^8x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^9x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^12x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^14x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^16x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^20x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^24x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^40x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace('^78x', '', regex = True)

go_clean['GENE'] = go_clean['GENE'].str.replace(' 2x', '', regex = True)
go_clean['GENE'] = go_clean['GENE'].str.replace(' 3x', '', regex = True)

## drop duplicates

In [ ]:
go_clean = go_clean.drop_duplicates()

## drop missing

In [ ]:
go_clean = go_clean.dropna()

## remove spaces

In [ ]:
go_clean['GENE'] = go_clean['GENE'].str.strip()

# map ENSIDs to gene names for gene scores

## map

In [ ]:
meta_gene_map = gene_map[gene_map['ENS_ID'].isin(meta_gene_list['GENE'])]
print(len(meta_gene_map.index))
print(len(meta_gene_map['ENS_ID'].unique()))
print(len(meta_gene_map['GENE'].unique()))
meta_gene_map.head()

## investigate dups

meta without finngen notes
- these are MANE Select Plus Clinical: Representative transcript from RefSeq & GENCODE
- PDE8B coordinates: chr5:77210680-77428256 (ENSG00000113231 overlaps more closely)
- CAST coordinates: chr5:96662376-96774683 (ENSG00000310517 overlaps more closely)
- LINC03023 coordinates: chr8:20953633-20968938 (ENSG00000253147 overlaps more closely)
- SPATA13 coordinates: chr13:24160720-24307069 (ENSG00000182957 overlaps more closely)
- PDE4C coordinates: chr19: chr19:18207965-18226438 (ENSG00000105650 overlaps more closely)

In [ ]:
meta_gene_map_dups = gene_map[gene_map['GENE'].isin(meta_gene_map['GENE'][meta_gene_map['GENE'].duplicated()])]
meta_gene_map_dups

## fix duplicates

In [ ]:
meta_gene_map_no_dup = meta_gene_map[~meta_gene_map['GENE'].isin(meta_gene_map_dups['GENE'])]
print(len(meta_gene_map.index))
print(len(meta_gene_map_no_dup.index))

In [ ]:
# no finngen meta fix
meta_gene_map_dup_fixed = meta_gene_map_dups[meta_gene_map_dups['ENS_ID'].isin(['ENSG00000113231',
                                                                                'ENSG00000310517',
                                                                                'ENSG00000253147',
                                                                                'ENSG00000182957',
                                                                                'ENSG00000105650'])]
meta_gene_map_dup_fixed

In [ ]:
meta_gene_map_final = pd.concat([meta_gene_map_no_dup, meta_gene_map_dup_fixed], axis = 0)
print(len(meta_gene_map_final.index))

## subset

In [ ]:
meta_gene_map_final = meta_gene_map_final[['ENS_ID', 'GENE']]
meta_gene_map_final.head()

# map ENSIDs to gene names for RNAseq genes

## map
- 667 RNAseq genes did not map
- genes didn't map because they were retired/discontinued

In [ ]:
gene_map_rnaseq = gene_map[gene_map['ENS_ID'].isin(rnaseq_gene[0])]
print(len(rnaseq_gene.index))
print(len(gene_map_rnaseq.index))
print(len(gene_map_rnaseq['ENS_ID'].unique()))
print(len(gene_map_rnaseq['GENE'].unique()))
gene_map_rnaseq.head()

## subset

In [ ]:
gene_map_rnaseq = gene_map_rnaseq[['ENS_ID', 'GENE']]
gene_map_rnaseq.head()

# map genes to pathways

## gene scores

In [ ]:
gs_go = go_clean[go_clean['GENE'].isin(meta_gene_map_final['GENE'])]
gs_go['SOURCE'] = 'ADSP_GS'
print(len(gs_go.index))
print(len(gs_go['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(meta_gene_map_final.index))
print(len(gs_go['PATHWAY_ID'].unique())) 
gs_go.head()

## rnaseq

In [ ]:
rs_go = go_clean[go_clean['GENE'].isin(gene_map_rnaseq['GENE'])]
rs_go['SOURCE'] = 'ROSMAP_RS'
print(len(rs_go.index))
print(len(rs_go['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(gene_map_rnaseq.index))
print(len(rs_go['PATHWAY_ID'].unique())) 
rs_go.head()

In [ ]:
msbb_rs_go = go_clean[go_clean['GENE'].isin(msbb_rnaseq_gene['GENE'])]
msbb_rs_go['SOURCE'] = 'MSBB_RS'
print(len(msbb_rs_go.index))
print(len(msbb_rs_go['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(msbb_rnaseq_gene.index))
print(len(msbb_rs_go['PATHWAY_ID'].unique())) 
msbb_rs_go.head()

## methylation

In [ ]:
ma_go = go_clean[go_clean['GENE'].isin(methyl_gene['RefGene'])]
ma_go['SOURCE'] = 'ROSMAP_MA'
print(len(ma_go.index))
print(len(ma_go['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(methyl_gene.index))
print(len(ma_go['PATHWAY_ID'].unique())) 
ma_go.head()

In [ ]:
msbb_ma_go = go_clean[go_clean['GENE'].isin(msbb_methyl_gene['GENE'])]
msbb_ma_go['SOURCE'] = 'MSBB_MA'
print(len(msbb_ma_go.index))
print(len(msbb_ma_go['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(msbb_methyl_gene.index))
print(len(msbb_ma_go['PATHWAY_ID'].unique())) 
msbb_ma_go.head()

## tmt protein

In [ ]:
msbb_tmt_go_gene = go_clean[go_clean['GENE'].isin(msbb_tmt_gene['GENE'])]
msbb_tmt_go_gene['SOURCE'] = 'MSBB_TP'
print(len(msbb_tmt_go_gene.index))
print(len(msbb_tmt_go_gene['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(msbb_tmt_gene.index))
print(len(msbb_tmt_go_gene['PATHWAY_ID'].unique())) 
msbb_tmt_go_gene.head()

## somoscan gene

In [ ]:
somoscan_go_gene = go_clean[go_clean['GENE'].isin(somoscan_gene['GENE'])]
somoscan_go_gene['SOURCE'] = 'ROSMAP_SP'
print(len(somoscan_go_gene.index))
print(len(somoscan_go_gene['GENE'].unique()))
print(len(go_clean['GENE'].unique()))
print(len(somoscan_gene['GENE'].unique()))
#print(len(somoscan_protein.index))
print(len(somoscan_go_gene['PATHWAY_ID'].unique())) 
somoscan_go_gene.head()

# merge

In [ ]:
all_gene = pd.concat([gs_go, rs_go, msbb_rs_go, ma_go, msbb_ma_go, msbb_tmt_go_gene, somoscan_go_gene], axis = 0)
all_gene = all_gene.groupby(['PATHWAY_NAME', 'PATHWAY_ID', 'GENE', 'RELATION'])['SOURCE'].agg(lambda x: ';'.join(sorted(set(x)))).reset_index()
print(len(all_gene.index))
all_gene.head()

# subset to pathway ID and gene and drop duplicates

In [ ]:
all_gene_sub = all_gene[['PATHWAY_ID', 'GENE']].drop_duplicates()
print(len(all_gene_sub.index))
print(len(all_gene_sub['PATHWAY_ID'].unique()))
print(len(all_gene_sub['GENE'].unique()))

# filter to ref genes

In [ ]:
all_gene_ref = all_gene[all_gene['GENE'].isin(ref['GENE'])]
print(len(all_gene_ref.index))
print(len(all_gene_ref['PATHWAY_ID'].unique()))
print(len(all_gene_ref['GENE'].unique()))

In [ ]:
all_gene_ref_sub = all_gene_sub[all_gene_sub['GENE'].isin(ref['GENE'])]
print(len(all_gene_ref_sub.index))
print(len(all_gene_ref_sub['PATHWAY_ID'].unique()))
print(len(all_gene_ref_sub['GENE'].unique()))

## pathway maps

In [ ]:
all_gene.to_csv('pathway_score/pathway_annotation/go/AD_KMI.ADSP.ROSMAP.all_omics.MSBB.all_omics.go.gene_to_pathway.pathway_mapping.txt',
                sep = '\t',
                index = None)

In [ ]:
all_gene_sub.to_csv('pathway_score/pathway_annotation/go/AD_KMI.ADSP.ROSMAP.all_omics.MSBB.all_omics.go.gene_to_pathway.no_duplicates.pathway_mapping.txt',
                    sep = '\t',
                    index = None)

In [ ]:
all_gene_ref.to_csv('pathway_score/pathway_annotation/go/AD_KMI.ADSP.ROSMAP.all_omics.MSBB.all_omics.GRCh38.113.refseq.exp_validated.go.gene_to_pathway.pathway_mapping.txt',
                    sep = '\t',
                    index = None)

In [ ]:
all_gene_ref_sub.to_csv('pathway_score/pathway_annotation/go/AD_KMI.ADSP.ROSMAP.all_omics.MSBB.all_omics.GRCh38.113.refseq.exp_validated.go.gene_to_pathway.no_duplicates.pathway_mapping.txt',
                        sep = '\t',
                        index = None)